In [1]:
#importing library
from langchain_groq import ChatGroq
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.output_parsers import StrOutputParser,PydanticOutputParser
from pydantic import BaseModel,Field
import requests
from langchain.tools import tool
from langchain_core.tools import StructuredTool,BaseTool
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
#creating an object of grog model class.
model = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002251F8B4190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002251F9C1870>, model_name='llama-3.1-8b-instant', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
model.invoke('hello')

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.005806151, 'prompt_time': 0.005550967, 'queue_time': 0.046201753, 'total_time': 0.011357118}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c40956ddc4', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--3a60c12f-aee4-4146-b6fd-2050b710ecdf-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

In [4]:
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate

In [5]:
# from langchain_tavily import TavilySearch

# search_tool = TavilySearch(
#     max_results=5,
#     topic="general"
# )

# # Run your query
# result = search_tool.invoke({"query": "top news in CNN"})
# print(result)


In [6]:
#creating an object of duckduck search tool.
#as we know that search tool is a runnable futhure we can used in chains or invoke
search_tool = DuckDuckGoSearchRun()


# result = search_tool.invoke("top news in CNN")
# result

# using built in agent class from langchain

```
2. from langchain.agents import create_react_agent
Ye function ek ReAct agent banata hai.
ReAct = Reasoning + Acting framework (paper se inspired).
Agent ko LLM + tools + prompt ke sath wrap karta hai taaki wo “reasoning steps likhe → tool call kare → final answer de”.


🔹 3. from langchain.agents import AgentExecutor
AgentExecutor = runner jo agent ko actually execute karta hai.
Ye input leta hai, reasoning + tool calls orchestrate karta hai, aur final output return karta hai.
```

In [7]:
#ek central prompt store hai jo LangChain ne banaya hai.
from langchain import hub #using this class we can get or pull standard reAct agaent prompt.
from langchain.agents import create_react_agent,AgentExecutor


# prompt hub 

In [8]:
#step 2: pulling the ReAct prompt(built in prompt) from langchain hub  
prompt = hub.pull("hwchase17/react")
prompt

c:\anaconda\envs\hf-gpu\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

# create the reAct agent

In [9]:
#step 3: create the reAct agent manually with the pulled prompt
#agent what it will do :- it will think/reasoning plan which tool will be used for Task
agent = create_react_agent(
    llm=model,
    prompt=prompt,
    tools=[search_tool]
)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'duckduckgo_search - A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.', 'tool_names': 'duckduckgo_search'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation c

# wrap it with agentExecutor 

In [10]:
#step 4: wrap it with agentExecutor class
#agentExecutor will execute the task
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True
    
)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'duckduckgo_search - A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.', 'tool_names': 'duckduckgo_search'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the ac

# invoke

In [11]:
result = agent_executor.invoke({'input': "show me summary 3 ways to reach goa from mumbai fast dont take to much time"})
result



> Entering new AgentExecutor chain...
Thought: To find the fastest ways to reach Goa from Mumbai, I should search for reliable travel information sources.
Action: duckduckgo_search
Action Input: "fastest ways to reach Goa from Mumbai"

c:\anaconda\envs\hf-gpu\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Aug 23, 2025 · Discover the best ways to reach Goa from Mumbai by flight, train, bus, and car. Easy travel tips, routes, and timings for a smooth journey. Dec 5, 2024 · Taking a Mumbai to Goa road trip is always an amazing experience. If you have no clue where to begin with, this quick guide you need to know. Feb 25, 2025 · Whether you're an adventure seeker, a nature lover, or a foodie, this ultimate guide will help you plan the perfect road trip from Mumbai to Goa! There are three main routes … Nov 4, 2024 · Planning a Mumbai to Goa road trip? Discover the best routes, top attractions, scenic stopovers, and essential tips to make it an adventure. 5 days ago · From Mumbai, the journey is simple and well connected by train, bus, car, and flight. Each option has its pros – trains and buses are budget-friendly, cars offer comfort and …Question: show me summary 3 ways to reach goa from mumbai fast dont take to much time
Thought: To find the fastest ways to reach Goa from Mumbai, I should 

c:\anaconda\envs\hf-gpu\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Aug 23, 2025 · Discover the best ways to reach Goa from Mumbai by flight, train, bus, and car. Easy travel tips, routes, and … May 6, 2025 · This article will give you some insight into the best ways to get from Mumbai to Goa. Goa is the smallest state in the country, but definitely one of the most loved. Dec 5, 2024 · Taking a Mumbai to Goa road trip is always an amazing experience. If you have no clue where to begin with, this … Feb 25, 2025 · Whether you're an adventure seeker, a nature lover, or a foodie, this ultimate guide will help you plan the … Nov 4, 2024 · Planning a Mumbai to Goa road trip? Discover the best routes, top attractions, scenic stopovers, and …Question: show me summary 3 ways to reach goa from mumbai fast dont take to much time
Thought: To find the fastest ways to reach Goa from Mumbai, I should search for reliable travel information sources.
Action: duckduckgo_search
Action Input: "fastest ways to reach Goa from Mumbai"

...

c:\anaconda\envs\hf-gpu\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Mumbai Tourism - Get Mumbai travel & tourism related information including tourist places, transport, nightlife, weather, photos, entertainment, shopping, festivals, maps, and attractions of ... Aug 18, 2025 · Mumbai rain Highlights: 7 people dead across Maharashtra, says CM Devendra Fadnavis Heavy monsoon rains lashed Mumbai on Monday morning, causing disruptions to traffic … Aug 19, 2025 · Mumbai Rains: Schools, colleges, offices, banks, OPEN or CLOSED today, August 19? Details here The India Meteorological Department (IMD) issued a red alert for Gujarat, … Aug 19, 2025 · Mumbai local train tracks under water, waterlogging between Ghatkopar and Dadar, central railway, western and harbour line delayed by 40 minutes According to regional news … Jun 19, 2025 · FASTag annual pass is not valid for two expressways, one is Meerut Expressway, other is…; how will 200 trips be counted in Rs 3000, know calculation here he FASTag annual …Question: show me summary 3 ways to reach goa from mumbai 

c:\anaconda\envs\hf-gpu\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Feb 25, 2025 · A road trip from Mumbai to Goa is one of the most thrilling travel experiences in India, offering scenic views, delicious food, adventure, and relaxation. Whether you choose the … Apr 12, 2025 · Explore our comprehensive guide on the best trains from Mumbai to Goa. Get essential travel tips and make your journey unforgettable. 4 days ago · Whether you're looking for government bus booking or specific services like RTC online booking, redBus makes it easy to access the best bus travel options for your needs. … Apr 19, 2025 · Plan your Mumbai to Goa ferry trip with the latest info on schedules, ticket prices, and travel tips. Discover a scenic and convenient way to explore Goa by sea 1 day ago · The average bus ticket price from Mumbai to Goa is approximately INR 550. With redBus, you can access exclusive offers, discounts, and deals, ensuring you get the cheapest …Thought: I now know the final answer
Final Answer: The fastest ways to reach Goa from Mumbai are by flight, t

{'input': 'show me summary 3 ways to reach goa from mumbai fast dont take to much time',
 'output': 'The fastest ways to reach Goa from Mumbai are by flight, train, and bus. Flight is the fastest option, taking approximately 1 hour. Train and bus options take around 8-10 hours, depending on the route and type of service.'}

In [12]:
print(result["output"])

The fastest ways to reach Goa from Mumbai are by flight, train, and bus. Flight is the fastest option, taking approximately 1 hour. Train and bus options take around 8-10 hours, depending on the route and type of service.


```
ReAct when to use:- jab websearch, database lookup karna tab we can be used

react mein  thought-->action--->observation loop hota based on the user query

agaentexecutor will run the react loop

```